# SOPRANO: a Python library for generation, manipulation and analysis of large batches of crystalline structures
by Simone Sturniolo

```
      _
    /|_|\   
   / / \ \  
  /_/   \_\  
  \ \   / /  
   \ \_/ /  
    \|_|/  

```
    
Developed within the CCP-NC project. Copyright STFC 2016


### TUTORIAL 5 - NMR Properties: using Soprano for working with .magres files

In [1]:
# Basic imports
import os, sys
sys.path.insert(0, os.path.abspath('..')) # This to add the Soprano path to the PYTHONPATH
                                          # so we can load it without installing it

In [2]:
# Other useful imports

import numpy as np

import ase
from ase import io as ase_io

## 1 - LOADING MAGRES FILES

ASE now is compatible with the Magres file format as well. If your version of ASE does not support loading
Magres files, try updating it or download the bleeding edge version from GitLab:

https://gitlab.com/ase/ase

In [3]:
C2H5O = ase_io.read('tutorial_data/ethanol.magres')

## 2 - LABELS AND INDICES

Labels and indices used in the magres file are stored in their own arrays and can be easily accessed and used to 
select atoms.

In [4]:
labels = C2H5O.get_array('labels')
indices = C2H5O.get_array('indices')

# Select only the protons' indices
H_i = np.where(labels == 'H')
H = C2H5O[H_i]
# Select only C2
C2_i = np.where((labels == 'C')*(indices == 2))
C2 = C2H5O[C2_i]

print "There are {0} hydrogen atoms".format(len(H))
print "The C2 atom is positioned at x = {0}, y = {1}, z = {2} Ang".format(*C2.get_positions()[0])

There are 6 hydrogen atoms
The C2 atom is positioned at x = -0.247915, y = 1.164107, z = 0.937396 Ang


In [6]:
# One can also recreate the Jmol-style labels if needed
jmol_labels = ["{0}_{1}".format(l, i) for l, i in zip(labels, indices)]
print "The labels are {0}".format(', '.join(jmol_labels))

The labels are H_1, H_2, H_3, H_4, H_5, H_6, C_1, C_2, O_1


## 3 - MAGNETIC SHIELDINGS AND CHEMICAL SHIFTS

All the NMR tensors stored in the original .magres file are saved as arrays in the Atoms object and can be accessed directly. However, Soprano also provides a set of properties to express the tensors in the form of parameters useful to compute the spectrum.

In [11]:
from soprano.properties.nmr import *

# Isotropy, Anisotropy and Asymmetry (Haeberlen convention)
iso = MSIsotropy.get(C2H5O)
aniso = MSAnisotropy.get(C2H5O)
asymm = MSAsymmetry.get(C2H5O)

print 'Label\tIsotropy\tAnisotropy\tAsymmetry'
for i, jl in enumerate(jmol_labels):
    print '{0}\t{1:.2f} ppm\t{2:.2f} ppm\t{3:.2f}'.format(jl, iso[i], aniso[i], asymm[i])

Label	Isotropy	Anisotropy	Asymmetry
H_1	29.59 ppm	8.94 ppm	0.14
H_2	30.26 ppm	8.19 ppm	0.21
H_3	30.10 ppm	7.29 ppm	0.06
H_4	26.98 ppm	8.17 ppm	0.94
H_5	27.39 ppm	-7.12 ppm	0.93
H_6	31.98 ppm	14.12 ppm	0.45
C_1	156.47 ppm	33.80 ppm	0.70
C_2	109.86 ppm	70.25 ppm	0.41
O_1	268.03 ppm	-51.38 ppm	0.98


In [9]:
# Span and skew
span = MSSpan.get(C2H5O)
skew = MSSkew.get(C2H5O)

print 'Label\tSpan\t\tSkew'
for i, jl in enumerate(jmol_labels):
    print '{0}\t{1:.2f} ppm\t{2:.2f}'.format(jl, span[i], skew[i])

Label	Span		Skew
H_1	9.36 ppm	-0.82
H_2	8.76 ppm	-0.74
H_3	7.43 ppm	-0.92
H_4	10.72 ppm	-0.05
H_5	9.33 ppm	0.05
H_6	16.25 ppm	-0.48
C_1	41.73 ppm	-0.24
C_2	79.95 ppm	-0.51
O_1	68.22 ppm	0.01
